In [ ]:
%matplotlib inline
import os
from datetime import datetime
import json
import pickle
import math
import utils
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD

vgg16 = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg16.summary()

In [ ]:
len(vgg16.layers)

In [ ]:
# vgg16モデル改造用の関数
def build_transfer_model(vgg16):
  model = Sequential(vgg16.layers)

  for layer in model.layers[:15]:
    layer.trainable = False
  
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  return model

model = build_transfer_model(vgg16)

In [ ]:
# len(model.layers)

In [ ]:
# 最適化アルゴリズムはSGD
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(lr=1e-4, momentum=0.9),
    metrics=['accuracy']
)
print(len(model.layers))
model.summary()

In [ ]:
# 画像ロード用イテレータ
idg_train = ImageDataGenerator(
    rescale=1/255.,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

In [ ]:
# 訓練用データ
img_itr_train = idg_train.flow_from_directory(
    '//content/drive/MyDrive/Colab Notebooks/vgg16/img/image/train',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary'
)

# 検証用データ
img_itr_validation = idg_train.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/vgg16/img/image/test',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary'
)

In [ ]:
# モデル保存用ディレクトリ
model_dir = os.path.join(
    '/content/drive/MyDrive/Colab Notebooks/vgg16/models',
    datetime.now().strftime('%y%m%d_%H%M')
)

os.makedirs(model_dir, exist_ok=True)

dir_weights = os.path.join(model_dir, 'weight')
os.makedirs(dir_weights, exist_ok=True)

In [ ]:
# モデル保存
model_json = os.path.join(model_dir, 'model.json')
with open(model_json, 'w') as f:
  json.dump(model.to_json(), f)

model_classes = os.path.join(model_dir, 'classes.pkl')
with open(model_classes, 'wb') as f:
  pickle.dump(img_itr_train.class_indices, f)

In [ ]:
# 1epochあたりのバッチ数計算
batch_size = 16
steps_per_epoch = math.ceil(
    img_itr_train.samples/batch_size
)

validation_steps = math.ceil(
    img_itr_validation.samples/batch_size
)

In [ ]:
# callbackを設定
cp_filepath = os.path.join(dir_weights, 'ep_{epoch:02d}_ls_{loss:.1f}.h5')
cp = ModelCheckpoint(
    cp_filepath,
    monitor='loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='auto',
    period=5
)

csv_filepath = os.path.join(model_dir, 'loss.csv')
csv = CSVLogger(csv_filepath, append=True)

In [ ]:
# 学習
n_epoch = 30

history = model.fit_generator(
    img_itr_train,
    steps_per_epoch=steps_per_epoch,
    epochs=n_epoch,
    validation_data=img_itr_validation,
    validation_steps=validation_steps,
    callbacks = [cp, csv]
)

In [ ]:
# 予測
test_data_dir = '/content/drive/MyDrive/Colab Notebooks/vgg16/img/image/pre'
x_test, true_labels = utils.load_random_imgs(
    test_data_dir,
    seed=1
)

print(x_test.shape)
print(true_labels)

# .jpgじゃないとエラー
x_test_preproc = preprocess_input(x_test.copy())/255.
probs = model.predict(x_test_preproc)

probs

In [ ]:
#評価
utils.show_test_samples(
    x_test, probs,
    img_itr_train.class_indices,
    true_labels
)

学習枚数を多くする
flickr等で